# Automatic Web-Scraping

In [76]:
# INSTALL TOR,BS4,REQUESTS,SOCKS,STEM,SOCKSPROXY,OS
!pip install requests bs4 stem requests[socks] socks urllib3[socks] tor
import os
import requests
from bs4 import BeautifulSoup
import socks
import socket
import urllib.request
import os
import time
import colorama
from colorama import Fore, Back, Style
from bs4 import BeautifulSoup


Defaulting to user installation because normal site-packages is not writeable

In [79]:


class Tor:
    def __init__(self):
        self.save_response = False
        self.ip = '127.0.0.1'
        self.port = 9050
        self.control_port = 9051
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        self.session = requests.session()
        self.session.proxies = {
            'http': 'socks5://{}:{}'.format(self.ip, self.port),
            'https': 'socks5://{}:{}'.format(self.ip, self.port)
        }
        # Startup Message
        print('-------------------------------')
        print(Fore.YELLOW + 'Starting Tor...')
        print(Fore.GREEN + 'Public IP: ' + Fore.RED + self.get_ip())


    def renew_tor(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(('127.0.0.1', 9051))
            print('-------------------------------')
            print(Fore.GREEN + 'Renewing IP...')
            TOR_PASSWORD = os.environ.get('TOR_PASSWORD') or 'my_password'
            s.send('AUTHENTICATE "{}"\r\n'.format(TOR_PASSWORD).encode())
            response = s.recv(128)
            if response != b'250 OK\r\n':
                print(response)
                raise ValueError('Unexpected response')
            s.send("signal NEWNYM\r\n".encode())
            response = s.recv(128)
            if response != b'250 OK\r\n':
                print(response)
                raise ValueError('Unexpected response')
        self.session = requests.session()
        self.session.proxies = {
            'http': 'socks5://{}:{}'.format(self.ip, self.port),
            'https': 'socks5://{}:{}'.format(self.ip, self.port)
        }            
        
    
    
    def get(self, url):
        if self.save_response:
            response = self.session.get(url, headers=self.headers).text
            self.save_page(response)
            return response
        else:
            return self.session.get(url, headers=self.headers).text
        
    
    def get_ip(self):
        return self.get('http://httpbin.org/ip')
    
    def save_page(self,response):
        with open('responses/'+str(time.time())+'.html','w',encoding="utf-8") as f:
            f.write(response)
    

In [ ]:
# Example

# Example
colorama.init(autoreset=True)
print(Fore.RED + 'Instanciating Tor')
tor = Tor()
# Make a request
print(Fore.GREEN + 'Making Request...')
response = tor.get('https://www.google.com')
# Renew IP
print(Fore.GREEN + 'Renewing IP...')
tor.renew_tor()
# Get IP
print(Fore.GREEN + 'Getting IP...')
print(Fore.RED + tor.get_ip())
# Make a request
print(Fore.GREEN + 'Making Request...')
tor.save_response = True
response = tor.get('https://www.google.com')
